In [17]:
import importlib
import numpy as np

import umap
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits

import matplotlib.pyplot as plt

from bayes_opt import BayesianOptimization

import AmbReducer as ambr

from snc.snc import SNC
importlib.reload(ambr)

<module 'AmbReducer' from '/home/hj/cluster-ambiguity/src/clams/AmbReducer.py'>

In [33]:
digits = load_digits().data

method = "tsne"

def get_loss(**kwargs):
	if method == "tsne":
		emb = TSNE(n_components=2, perplexity=kwargs["perplexity"], random_state=0).fit_transform(digits)
	elif method == "umap":
		emb = umap.UMAP(n_components=2, n_neighbors=int(kwargs["n_neighbors"]), min_dist=kwargs["min_dist"], random_state=0).fit_transform(digits)
	emb=digits
	metrics = SNC(digits, emb, iteration=300)
	metrics.fit()
	stead, cohev = metrics.steadiness(), metrics.cohesiveness()
	return (stead * cohev) / (stead + cohev)

optimizer = BayesianOptimization(
	f=get_loss,
	pbounds={
		"perplexity": (5, 500)
	},
	random_state=0,
	verbose=2
)

optimizer.maximize(init_points=5, n_iter=25)

max_hyperparameter = optimizer.max["params"]
if method == "tsne":
	emb = TSNE(n_components=2, perplexity=max_hyperparameter["perplexity"], random_state=0).fit_transform(digits)
elif method == "umap":
	emb = umap.UMAP(n_components=2, n_neighbors=int(max_hyperparameter["n_neighbors"]), min_dist=max_hyperparameter["min_dist"], random_state=0).fit_transform(digits)
		


plt.scatter(emb[:, 0], emb[:, 1], s=0.2)


|   iter    |  target   | perple... |
-------------------------------------


TypeError: float() argument must be a string or a number, not 'Bunch'

In [15]:
amb = ambr.AmbReducer(method="tsne", metric="snc", threshold_loss=0.05, S=3.0, verbose=1)
amb.fit(raw, emb)

new_emb = amb.optimize()

initial loss 0.38164573025594417
|   iter    |  target   | perple... |
-------------------------------------
new loss 0.4049441828986467
1 - ambiguity 0.5129631170093489
new loss 0.40644616746751006
1 - ambiguity 0.42588232444926266
new loss 0.40466145188893954
1 - ambiguity 0.4218615634308662
new loss 0.41473238751293
1 - ambiguity 0.48273265741194005
new loss 0.4183137151471117
1 - ambiguity 0.6185463654659947
|  5        |  0.6185   |  214.7    |
new loss 0.40375403056800785
1 - ambiguity 0.48547842645914185
new loss 0.424039934196632
1 - ambiguity 0.5839126615287064
new loss 0.3917027153449805
1 - ambiguity 0.47473713247975535
new loss 0.3821939494365114
1 - ambiguity 0.37754867031403316
new loss 0.41879805583233864
1 - ambiguity 0.5685893385360639
new loss 0.4194417684427686
1 - ambiguity 0.617718595010956
new loss 0.4253381551507956
1 - ambiguity 0.604473884300872
new loss 0.41941487108092146
1 - ambiguity 0.711184239359136
|  13       |  0.7112   |  159.7    |
new loss 0.4236355

KeyboardInterrupt: 

In [ ]:

plt.scatter(new_emb[:, 0], new_emb[:, 1], s=0.2)